# Estimating Trip Mode Choice

This notebook illustrates how to re-estimate a single model component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import os
import larch  # !conda install larch -c conda-forge # for estimation
import pandas as pd

We'll work in our `test` directory, where ActivitySim has saved the estimation data bundles.

In [2]:
os.chdir('test')

# Load data and prep model for estimation

In [3]:
modelname = "trip_mode_choice"

from activitysim.estimation.larch import component_model
model, data = component_model(modelname, return_data=True)

/Users/jeffnewman/OneDrive - Cambridge Systematics/Git/activitysim/activitysim/estimation/larch/mode_choice.py:126: DtypeWarning: Columns (411) have mixed types.Specify dtype option on import or set low_memory=False.
  return mode_choice_model(


# Review data loaded from the EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

### Coefficients

In [4]:
data.coefficients

,value,constrain
coefficient_name,,
coef_one,1.0000,T
coef_nest_root,1.0000,T
coef_nest_AUTO,0.7200,T
coef_nest_AUTO_DRIVEALONE,0.3500,T
coef_nest_AUTO_SHAREDRIDE2,0.3500,T
...,...,...
coef_walk_transit_ASC_walk_eatout,1.3504,F
walk_express_penalty,10.0000,T
adjust_tnc_shared,30.0000,T


#### Utility specification

In [5]:
data.spec

,Label,Description,Expression,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,...,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
0,util_DRIVEALONEFREE_Unavailable,DRIVEALONEFREE - Unavailable,sov_available == False,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,util_DRIVEALONEFREE_Unavailable_for_zero_auto_...,DRIVEALONEFREE - Unavailable for zero auto hou...,auto_ownership == 0,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,util_DRIVEALONEFREE_Unavailable_for_persons_le...,DRIVEALONEFREE - Unavailable for persons less ...,age < 16,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,util_DRIVEALONEFREE_Unavailable_for_joint_tours,DRIVEALONEFREE - Unavailable for joint tours,is_joint == True,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,util_DRIVEALONEFREE_Unavailable_if_didnt_drive...,DRIVEALONEFREE - Unavailable if didn't drive t...,is_atwork_subtour & ~work_tour_is_SOV,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,util_Walk_not_available_for_long_distances,Walk not available for long distances,@df.tour_mode_is_walk & (od_skims['DISTWALK'] ...,NaN,NaN,NaN,NaN,NaN,NaN,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376,util_Bike_not_available_for_long_distances,Bike not available for long distances,@df.tour_mode_is_walk & (od_skims['DISTBIKE'] ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377,util_origin_density_index,Origin density index,@origin_density_applied*(origin_density_index_...,NaN,NaN,NaN,NaN,NaN,NaN,coef_ivt,...,coef_ivt,coef_ivt,NaN,NaN,NaN,NaN,NaN,NaN,coef_ivt,coef_ivt
378,util_walk_express_penalty,Walk-express penalty for intermediate stops,@walk_express_penalty * ~(df.first_trip | df.f...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Chooser data

In [6]:
data.chooser_data

,trip_id,model_choice,override_choice,util_DRIVEALONEFREE_Unavailable,util_DRIVEALONEFREE_Unavailable_for_zero_auto_households,util_DRIVEALONEFREE_Unavailable_for_persons_less_than_16,util_DRIVEALONEFREE_Unavailable_for_joint_tours,util_DRIVEALONEFREE_Unavailable_if_didnt_drive_to_work,util_DRIVEALONEFREE_In_vehicle_time,util_DRIVEALONEFREE_Terminal_time,...,drive_lrf_available_inbound,drive_express_available_outbound,drive_express_available_inbound,drive_heavyrail_available_outbound,drive_heavyrail_available_inbound,drive_commuter_available_outbound,drive_commuter_available_inbound,walk_ferry_available,drive_ferry_available,override_choice_code
tour_id,,,,,,,,,,,,,,,,,,,,,
6812,54497,WALK,WALK,0.0,1.0,0.0,0.0,0.0,2.72,7.72508,...,False,False,False,False,False,False,False,False,False,7
6812,54501,WALK,WALK,0.0,1.0,0.0,0.0,0.0,2.76,7.72508,...,False,False,False,False,False,False,False,False,False,7
8110,64881,WALK,WALK,0.0,1.0,0.0,0.0,0.0,4.43,17.09892,...,False,False,False,False,False,False,False,False,False,7
8110,64885,WALK,WALK,0.0,1.0,0.0,0.0,0.0,4.43,17.09892,...,False,False,False,False,False,False,False,False,False,7
11013,88105,DRIVEALONEFREE,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,10.53,9.08660,...,False,False,False,False,False,False,False,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309796968,2478375745,SHARED3FREE,SHARED3FREE,0.0,0.0,0.0,0.0,0.0,8.84,6.83436,...,False,False,False,False,False,False,False,False,False,5
309796968,2478375749,SHARED3FREE,SHARED3FREE,0.0,0.0,0.0,0.0,0.0,8.50,6.83436,...,False,False,False,False,False,False,False,False,False,5
309796969,2478375753,DRIVEALONEFREE,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,4.00,5.00346,...,False,False,False,False,False,False,False,False,False,1


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [7]:
model.load_data()
model.doctor(repair_ch_av='-')
model.loglike()

req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
problem: chosen-but-not-available (2 issues)
problem: low-variance-data-co (1 issues)
problem: chosen-but-not-available (2 issues)
problem: low-variance-data-co (1 issues)
problem: chosen-but-not-available (1 issues)
problem: low-variance-data-co (1 issues)
problem: chosen-but-no

-10094.898223413013

In [8]:
model.maximize_loglike(method='SLSQP', options={"maxiter": 1000})

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
-999,-999.000000,-999.0000,-999.0,-999.0,-999.0,1,,-999.000000
1,1.000000,1.0000,1.0,1.0,1.0,1,,1.000000
coef_age010_trn,0.360551,0.0000,0.0,NaN,NaN,0,,0.360551
coef_age1619_da,0.344530,0.0000,0.0,NaN,NaN,0,,0.344530
coef_age16p_sr,-0.151886,0.0000,0.0,NaN,NaN,0,,-0.151886
...,...,...,...,...,...,...,...,...
coef_sr3p_ASC_sr3p_othdiscr,0.844944,0.7398,0.0,NaN,NaN,0,,0.844944
coef_sr3p_ASC_walk_othdiscr,-5.841028,-0.5272,0.0,NaN,NaN,0,,-5.841028
coef_walk_transit_ASC_sr2_othdiscr,-1.232224,-2.7590,0.0,NaN,NaN,0,,-1.232224
coef_walk_transit_ASC_sr3p_othdiscr,-1.199264,-2.5126,0.0,NaN,NaN,0,,-1.199264


┣                x: -999                                  -999.000000
┃                   1                                        1.000000
┃                   coef_age010_trn                          0.360551
┃                   coef_age1619_da                          0.344530
┃                   coef_age16p_sr                          -0.151886
┃                                                             ...    
┃                   coef_sr3p_ASC_sr3p_othdiscr              0.844944
┃                   coef_sr3p_ASC_walk_othdiscr             -5.841028
┃                   coef_walk_transit_ASC_sr2_othdiscr      -1.232224
┃                   coef_walk_transit_ASC_sr3p_othdiscr     -1.199264
┃                   coef_walk_transit_ASC_walk_othdiscr     -0.980497
┃                   Length: 272, dtype: float64
┣          loglike: -7541.342641931288
┣        d_loglike: -999                                   0.000000
┃                   1                                      0.000000
┃                   coef_age010_trn                        0.000007
┃                   coef_age1619_da                       -0.000020
┃                   coef_age16p_sr                        -0.001532
┃                                                            ...   
┃                   coef_sr3p_ASC_sr3p_othdiscr           -0.000008
┃                   coef_sr3p_ASC_walk_othdiscr            0.000050
┃                   coef_walk_transit_ASC_sr2_othdiscr     0.000009
┃                   coef_walk_transit_ASC_sr3p_othdiscr    0.000023
┃                   coef_walk_transit_ASC_walk_othdiscr   -0.000047
┃                   Length: 272, dtype: float64
┣              nit: 110
┣             nfev: 255
┣             njev: 110
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=132, microseconds=823819)
┣           method: 'SLSQP'
┣          n_cases: 13279
┣ iteration_number: 110
┣          logloss: 0.5679149515725046

### Estimated coefficients

In [9]:
model.parameter_summary()

,Value,Null Value
-999,-999.,-999.00
1,1.00,1.00
coef_age010_trn,0.361,0.00
coef_age1619_da,0.345,0.00
coef_age16p_sr,-0.152,0.00
coef_bike_ASC_rh,-7.00,0.00
coef_bike_ASC_walk_work,-2.66,0.00
coef_drive_transit_ASC_commuter_work,0.427,0.00
coef_drive_transit_ASC_express_work,-0.355,0.00
coef_drive_transit_ASC_ferry_work,0.528,0.00


# Output Estimation Results

In [15]:
from activitysim.estimation.larch import update_coefficients
result_dir = data.edb_directory/"estimated"
update_coefficients(
    model, data, result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [18]:
for m in model:
    m.to_xlsx(
        result_dir/f"{m.title}_{modelname}_model_estimation.xlsx", 
        data_statistics=False,
    )

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [19]:
pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_one,1.000000,T
1,coef_nest_root,1.000000,T
2,coef_nest_AUTO,0.720000,T
3,coef_nest_AUTO_DRIVEALONE,0.350000,T
4,coef_nest_AUTO_SHAREDRIDE2,0.350000,T
...,...,...,...
309,coef_walk_transit_ASC_walk_eatout,0.166429,F
310,walk_express_penalty,10.000000,T
311,adjust_tnc_shared,30.000000,T
312,coef_origin_density_applied_work_univ_school,0.000000,T
